In [ ]:
#llm RAG 플러그인 설정(임베딩)

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 필요한 라이브러리 import 수정
import pandas as pd
from langchain_core.documents import Document # LangChain의 기본 문서 형식을 import
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [ ]:
def setup_rag_pipeline_chroma(file_path: str, embedding_model_name: str, persist_directory: str):
    """
    CSV 파일로부터 ChromaDB RAG 파이프라인을 구축하고 검색기(Retriever)를 반환합니다.
    """
    # 1. Embed: 임베딩 모델 로드 (이전과 동일)
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
    print("✅ 임베딩 모델을 로드했습니다.")
    
    # 2. 벡터 스토어 생성 또는 로드
    # persist_directory에 DB가 이미 존재하면 새로 만들지 않고 바로 로드합니다.
    if os.path.exists(persist_directory):
        print("✅ 기존 ChromaDB를 로드합니다.")
        vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    else:
        print("⚠️ 기존 ChromaDB가 없으므로 새로 생성합니다.")
        # 1. Pandas로 CSV 파일 불러오기
        df = pd.read_csv(file_path, index_col=0)
        print(f"✅ Pandas로 CSV 파일을 불러왔습니다. (총 {len(df)}개 행)")

        # 2. 각 행을 자연어 문장으로 변환하고, LangChain 'Document' 객체로 만들기
        langchain_documents = []
        for timestamp, values in df.iterrows():
            # 직접 만드신 자연어 변환 로직
            sentence = f"'{timestamp}'의 "
            value_strings = [f"'{col}'는 {val}" for col, val in values.items()]
            sentence += ", ".join(value_strings)
            
            # 변환된 문장을 page_content로 하는 Document 객체 생성
            # metadata에 원본 시간 정보를 넣어두면 나중에 유용하게 사용할 수 있습니다.
            doc = Document(
                page_content=sentence,
                metadata={"timestamp": timestamp}
            )
            langchain_documents.append(doc)
        
        print(f"✅ {len(langchain_documents)}개의 자연어 문서를 생성했습니다.")
        
        # Store: ChromaDB를 생성하고 파일로 저장(persist)합니다.
        vectorstore = Chroma.from_documents(
            documents=langchain_documents, 
            embedding=embeddings,
            persist_directory=persist_directory # 이 경로에 DB 파일이 저장됩니다.
        )
        print(f"✅ ChromaDB를 생성하고 '{persist_directory}' 경로에 저장했습니다.")

    # 3. Retriever 생성 (이전과 완벽히 동일)
    retriever = vectorstore.as_retriever()
    print("✅ Retriever를 생성했습니다. RAG 파이프라인 준비 완료.")
    
    return retriever

In [ ]:
# --- RAG 파이프라인 실행 ---
CSV_FILE_PATH = 'RTDB_test.csv'
EMBEDDING_MODEL = 'nlpai-lab/KURE-v1'
CHROMA_DB_PATH = 'chroma_db' # DB 파일이 저장될 폴더 이름
# ChromaDB용 함수를 호출합니다.
rag_retriever = setup_rag_pipeline_chroma(CSV_FILE_PATH, EMBEDDING_MODEL, CHROMA_DB_PATH)

In [ ]:
#llm 메인 모델 설정

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key="GEMINI_API_KEY",
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


In [ ]:
from llama_cpp import Llama
import json
import os
from llama_cpp.llama_chat_format import Jinja2ChatFormatter

In [ ]:
# --- 2. Jinja 채팅 템플릿 파일에서 로드 (수정된 부분) ---
# 'lmalign_v1.jinja' 파일이 이 스크립트와 같은 디렉토리에 있는지 확인하세요.
jinja_template_path = "lmalign_v1.jinja"

if not os.path.exists(jinja_template_path):
    raise FileNotFoundError(
        f"'{jinja_template_path}' 파일을 찾을 수 없습니다. "
        "스크립트와 동일한 디렉토리에 파일을 위치시켜 주세요."
    )

with open(jinja_template_path, "r", encoding="utf-8") as f:
    jinja_template = f.read()

# --- 3. Jinja2ChatFormatter 인스턴스 생성 ---
chat_handler = Jinja2ChatFormatter(
    template=jinja_template,
    eos_token="<|eot_id|>",
    bos_token="<|begin_of_text|>"
)

In [ ]:
# --- 3. Llama 모델 로드 ---
# 다운로드한 모델과 파일에서 읽어온 Jinja 템플릿을 사용하여 모델을 초기화합니다.
llm = Llama(
  model_path=model_path,
  n_ctx=8192,         # Context window 사이즈
  n_gpu_layers=-1,    # GPU 가속 사용 (가능한 경우)
  verbose=False,
)

In [ ]:
# --- 4. 추론 실행 ---
# 서비스 설명 및 출력 통제 지시사항을 포함한 시스템 프롬프트 구성
#추가 프롬: - 반드시 JSON 포맷으로 응답

## 평가자 llm 시스템 프롬프트
evaluation_system_prompt = """
당신은 고도로 전문화된 특허 심사관이자 기술 분석가입니다.

당신의 유일한 임무는 [사용자 아이디어]와 [특허 문서 조각]을 입력받아, 두 내용의 기술적 유사성을 정밀하게 분석하고 그 결과를 보고하는 것입니다.

당신은 **절대** 일반적인 대화형 텍스트로 응답해서는 안 됩니다. 당신의 **유일한** 응답은 `evaluation_idea` 함수를 호출하는 것이어야 합니다.

---
### ⚖️ 평가 지침

당신은 다음의 엄격한 기준에 따라 [사용자 아이디어]와 [특허 문서 조각]을 비교하고, `evaluation_idea` 함수를 호출해야 합니다.

1.  **`eval_score` (0-100점 사이의 정수):**
    * `eval_score`는 **0점에서 100점 사이의 모든 정수(integer)**로 평가되어야 합니다. (예: 65점, 82점, 91점)
    * 아래의 점수 벤치마크는 **판단의 기준선(Benchmark)**입니다. 이 기준을 **참고**하여, 두 내용의 유사도에 가장 적합하다고 판단되는 **구체적인 정수 점수**를 도출하세요.

    * **0점 (기준):** [아이디어]와 [특허 조각]이 기술적으로 완전히 무관합니다.
    * **25점 (참고 기준):** 일부 키워드는 겹치나, 핵심 기술 원리나 해결하고자 하는 문제가 다릅니다.
    * **50점 (중간 기준):** 유사한 문제를 해결하려 하나, 사용된 기술적 접근 방식이나 핵심 구성이 상이합니다.
    * **75점 (참K 기준):** 핵심 기술 원리 및 접근 방식이 매우 유사하며, 일부 사소한 구성 요소나 적용 범위에서만 차이를 보입니다.
    * **100점 (기준):** 두 내용의 핵심 기술적 사상(발명의 원리)이 실질적으로 동일하거나 명백한 동등물입니다.

2.  **`reason` (문자열):**
    * 해당 `eval_score`를 부여한 명확하고 전문가적인 근거를 제시해야 합니다.
    * **[매우 중요]** [사용자 아이디어]의 **'어떤 개념'**이 [특허 문서 조각]의 **'어느 특정 부분(문구 인용 가능)'**과 구체적으로 어떻게 유사한지 (또는 다른지) 반드시 명시해야 합니다.
    * 추상적인 설명이 아닌, 구체적인 비교 분석 내용을 포함해야 합니다.
---

이제 [사용자 아이디어]와 [특허 문서 조각]을 바탕으로 분석을 수행하고, 즉시 `evaluation_idea` 함수를 호출하세요.
"""


In [ ]:
# --- 5. Function Calling 테스트 ---

# 5-1. 파이썬 함수 정의
# '-> float'는 이 함수가 숫자(float)를 반환한다는 것을 의미하는 타입 힌트입니다.
def calculate_average(val1: float, val2: float) -> float:
    """두 개의 숫자 값의 산술 평균을 계산하고, 그 숫자 값을 반환합니다."""
    try:
        average = (val1 + val2) / 2
        print(f"--- 'calculate_average' 호출됨 (val1={val1}, val2={val2}), 결과: {average} ---")
        return average # 👈 문장 대신 숫자 값을 직접 반환
    except TypeError:
        return 0.0 # 오류 발생 시 0.0 반환

# 5-2. 도구(함수) 목록 정의 (OpenAI tool-call 형식)
eval_tools = [
    {
        "type": "function",
        "function": {
            "name": "evaluation_idea",
            "description": "사용자의 아이디어와 특허 문서 조각의 유사도를 분석하여, 0-100점 사이의 점수와 그 근거를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "eval_score": {
                        "type": "integer",
                        "description": "사용자 아이디어와 특허 조각 간의 기술적 유사도 점수. 0 (완전히 무관함)에서 100 (기술적으로 동일함) 사이의 정수입니다.",
                        "minimum": 0,
                        "maximum": 100
                    },
                    "reason": {
                        "type": "string",
                        "description": "해당 점수를 부여한 구체적인 이유. 특허 조각의 어느 부분이 아이디어의 어떤 개념과 유사한지(또는 다른지) 명확히 짚어서 설명해야 합니다."
                    }
                },
                "required": ["eval_score", "reason"],
            },
        },
    }
]



In [ ]:
import re
import json

def execute_llm_turn(messages: list, max_depth: int = 5):
    """
    주어진 대화 기록(messages)을 바탕으로 LLM을 1회 호출하고,
    그 결과가 함수 호출이면 재귀적으로 자신을 다시 호출하는 핵심 함수.
    """
    # 1. 재귀 호출의 안전장치: 최대 호출 횟수를 초과하면 에러 메시지 반환
    if max_depth <= 0:
        return "최대 함수 호출 횟수를 초과하여 처리를 중단합니다."

    # 2. 주어진 messages로 LLM 추론 실행 (코드 중복 제거)
    prompt_response = chat_handler(messages=messages, tools=tools)
    response = llm.create_completion(
        prompt=prompt_response.prompt,
        max_tokens=2048,
        stop=prompt_response.stop,
        temperature=0.3
    )

    response = client.chat.completions.create(
    model="gemini-2.5-flash",
    reasoning_effort="medium",
    messages=[
        {"role": "system", "content": evaluation_system_prompt},
        {
            "role": "user",
            "content": f"[사용자 아이디어]:{user_idea}\n [특허 문서 조각]: {do}"
        }
    ],
    tools=eval_tools,
    tool_choice="auto"
)
    output_text = response['choices'][0]['text'].strip()

    # 3. LLM의 응답에서 함수 호출 패턴 확인
    match = re.search(r"<function=.*?</function>", output_text)

    # 4. 재귀의 분기점: 함수 호출이 있는지 여부
    if not match:
        # [재귀 종료 조건] 함수 호출이 없으면, 현재 텍스트가 최종 답변임
        print("🤖 AI 판단: 일반 답변 (재귀 종료)")
        return output_text
    else:
        # [재귀 호출 조건] 함수 호출이 있으면, 추가 처리 후 자신을 다시 호출
        print(f"🤖 AI 판단: 함수 호출 필요 (남은 호출 횟수: {max_depth-1})")
        function_call_string = match.group(0)
        
        try:
            # 4-1. 함수 실행 및 결과 얻기
            func_name = function_call_string.split('>')[0].split('=')[1]
            args_str = function_call_string.split('>')[1].split('<')[0]
            args_json = json.loads(args_str)
            
            result_data = None
            if func_name == "calculate_average":
                result_data = calculate_average(**args_json)
            
            # 4-2. 다음 재귀 호출을 위한 대화 기록 업데이트
            # 이전 기록에 LLM의 함수 호출 응답과 실제 함수 실행 결과를 추가
            updated_messages = messages + [
                {"role": "assistant", "content": output_text},
                {"role": "ipython", "content": str(result_data)}
            ]
            
            # 4-3. 업데이트된 대화 기록으로 자기 자신을 다시 호출 (재귀)
            return execute_llm_turn(messages=updated_messages, max_depth=max_depth - 1)

        except Exception as e:
            print(f"❌ 함수 호출 처리 중 오류 발생: {e}")
            return "함수를 실행하는 중에 오류가 발생했습니다."

def get_single_turn_response(user_query: str):
    """
    사용자의 초기 질문을 받아 재귀 함수의 실행을 시작하는 진입점 함수.
    """
    print(f"\n🚀 새 질문 처리 시작: \"{user_query}\"")
    # 1. 사용자의 첫 질문으로 초기 대화 기록 생성
    initial_messages = [{"role": "user", "content": user_query}]
    # 2. 재귀 함수를 호출하여 전체 프로세스 시작
    return execute_llm_turn(messages=initial_messages)

In [ ]:
def answer_with_rag(user_query: str):
    """
    사용자 질문에 대해 RAG를 통해 찾은 정보와 함께 LLM에게 답변을 요청하는 함수
    """
    print(f"\n--- 사용자 질문: {user_query} ---")
    
    # 1. Retrieve: RAG Retriever로 사용자 질문과 가장 관련 높은 문서를 검색합니다.
    retrieved_docs = rag_retriever.invoke(user_query)
    print(f"📚 {len(retrieved_docs)}개의 관련 문서를 찾았습니다.")

    # 2. 컨텍스트(Context) 생성: 검색된 문서들의 내용을 하나의 문자열로 합칩니다.
    context_data = "\n".join([f"- {doc.page_content}" for doc in retrieved_docs])
    '''
    print("--- 검색된 RAG 데이터 ---")
    print(context_data)
    print("-------------------------")
    '''
    # 3. 프롬프트 구성: 기존의 프롬프트 형식에 질문과 RAG 데이터를 결합합니다.
    # 이는 get_single_turn_response 함수 내부에서 처리되므로,
    # 해당 함수에 전달할 형태로 질문을 재구성합니다.
    final_query_for_llm = f"""
    아래 [데이터]를 참고하여 질문에 대답해주세요.
    
    [질문]
    {user_query}

    [데이터]
    {context_data}
    """
    
    # 4. Generate: 재구성된 질문을 기존에 만들었던 LLM 호출 함수에 전달합니다.
    final_answer = get_single_turn_response(final_query_for_llm)
    
    return final_answer

In [ ]:
# --- 실제 서비스 로직 예시 (Stateless) ---

# Case 1: 일반적인 파싱
print("--- Case 1: 함수 호출 질문 ---")
user_input_1 = ""
final_response_1 = answer_with_rag(user_input_1)
print("\n[최종 답변]")
print(final_response_1)